# Template Search

**Author**: Dr. Alexis Salas Burgos
**Laboratory**: Nanocell Lab
**Date**: Jun 2019
**Version**: 2.0

## Requirements

**Software**
* modeller
> export KEY_MODELLER=MODELIRANJE
> conda install -c salilab modeller
* mafft
> conda install -c biocore mafft 

**Python**
* biopython
> conda install biopython
* prody 
>pip install prody

## Method 01 with BLAST

In [2]:
def code2pirseq(code):
    """Función auxiliar para obtener el formato PIR que es el reconocido por Modeller desde un código de SwissProt"""
    from Bio import ExPASy
    from Bio import SeqIO
    spraw = ExPASy.get_sprot_raw(code)
    spinfo = SeqIO.read(spraw, "swiss")
    PDBs = [x[4:] for x in spinfo.dbxrefs if x.startswith('PDB:')]
    #print len(PDBs), PDBs
    
    # Name of the protein (a name without a white space is preferred)
    name     = str(spinfo.name)
    sequence = str(spinfo.seq)
    outname  = "%s.ali" % name
    # Formato del 
    with open(outname, "w") as f:
        f.write(">P1;%s\n" % name)
        f.write("sequence:%s: %d:A:%d: A:::0.00: 0.00\n" % (name, 1, len(sequence)) )
        f.write("%s*\n" % sequence)
    return name, outname, sequence

In [4]:
#Obtain name and sequence
spcode = "P11167"
name, outfile, sequence = code2pirseq(spcode)
print (outfile)

GTR1_RAT.ali


In [11]:
%more GTR1_RAT.ali

>P1;GTR1_RAT
sequence:GTR1_RAT: 1:A:492: A:::0.00: 0.00
MEPSSKKVTGRLMLAVGGAVLGSLQFGYNTGVINAPQKVIEEFYNQTWNHRYGESIPSTTLTTLWSLSVAIFSVGGMIGSFSVGLFVNRFGRRNSMLMMNLLAFVSAVLMGFSKLGKSFEMLILGRFIIGVYCGLTTGFVPMYVGEVSPTALRGALGTLHQLGIVVGILIAQVFGLDSIMGNADLWPLLLSVIFIPALLQCILLPFCPESPRFLLINRNEENRAKSVLKKLRGTADVTRDLQEMKEEGRQMMREKKVTILELFRSPAYRQPILIAVVLQLSQQLSGINAVFYYSTSIFEKAGVQQPVYATIGSGIVNTAFTVVSLFVVERAGRRTLHLIGLAGMAGCAVLMTIALALLEQLPWMSYLSIVAIFGFVAFFEVGPGPIPWFIVAELFSQGPRPAAVAVAGFSNWTSNFIVGMCFQYVEQLCGPYVFIIFTVLLVLFFIFTYFKVPETKGRTFDEIASGFRQGGASQSDKTPEELFHPLGADSQV*


In [12]:
# Blast contra PDB desde la Web
import Bio.Blast.NCBIWWW as NCBIWWW
blast_handle = NCBIWWW.qblast(program="blastp", database="pdb", sequence=sequence, expect=0.001)

In [13]:
"""Read the Blast output y extra desde los record el código PDB y la cadena similar"""
from Bio.Blast import NCBIXML
# lee el blast desde memoria
blast_records = NCBIXML.parse(blast_handle)
pdbcodes = []
chaincodes = []
for record in blast_records:
    for alignment in record.alignments:
        pdbcode = str(alignment.title).split("|")[3][:]
        chaincode = str(alignment.title).split("|")[4][0]
        #print alignment.title
        pdbcodes.append(pdbcode)
        chaincodes.append(chaincode)
        #for hsp in alignment.hsps:
            #print hsp.query
        #    #print hsp
        #    save_file.write('>%s\n' % (hsp.query))
print (pdbcodes, chaincodes)

['5EQG', '4PYP', '4ZW9', '5C65', '4YBQ', '4YB9', '4LDS', '4JA3', '4QIQ', '4GBY', '6N3I', '6H7D'] ['A', 'A', 'A', 'A', 'A', 'D', 'A', 'A', 'A', 'A', 'A', 'A']


In [23]:
# Option 1 descargar con prody
import prody
#Descomentar en caso de no poder realizar el BLAST
#pdbcodes = ['5EQG', '4PYP', '4ZW9', '5C65', '4YBQ']; chaincodes = ['A', 'A', 'A', 'A', 'A']
pdbsel = pdbcodes[:]
#Cambiar el folder
data_pdbsel = prody.fetchPDB(pdbsel, folder='.', compressed=False)

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 4yb9 downloaded (4yb9.pdb)
@> 4lds downloaded (4lds.pdb)
@> 4ja3 downloaded (4ja3.pdb)
@> 4qiq downloaded (4qiq.pdb)
@> 4gby downloaded (4gby.pdb)
@> 6n3i downloaded (6n3i.pdb)
@> 6h7d downloaded (6h7d.pdb)
@> PDB download via FTP completed (7 downloaded, 0 failed).


In [ ]:
# Option 2 descargar PDB con biopython

In [24]:
# Aquí se seleccionan sólo las cadenas
pdbsel = [x.lower() for x in pdbcodes[:]]
arregloPDBs = []
for i,p in enumerate(pdbsel):
    arr = (p, chaincodes[i])
    arregloPDBs.append(arr)
arregloPDBs

[('5eqg', 'A'),
 ('4pyp', 'A'),
 ('4zw9', 'A'),
 ('5c65', 'A'),
 ('4ybq', 'A'),
 ('4yb9', 'D'),
 ('4lds', 'A'),
 ('4ja3', 'A'),
 ('4qiq', 'A'),
 ('4gby', 'A'),
 ('6n3i', 'A'),
 ('6h7d', 'A')]

In [ ]:
# Alinear los PDB en una matriz de distancia
# Alinear secuencias con MAFFT

In [33]:
from modeller import *

env = environ()
aln = alignment(env)
for (pdb, chain) in arregloPDBs:
   m = model(env, file=pdb, model_segment=('FIRST:'+chain, 'LAST:'+chain))
   aln.append_model(m, atom_files=pdb, align_codes=pdb+chain)
print ("Malign")
aln.malign()

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL9v21}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v21}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4532674    4426.439     4.323

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4533202    4426.955     4.323
openf___224_> Open           ${MODINSTALL9v21}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4581802    4474.416     4.370
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL9v21}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4595102    4487.404     4.382
openf___224_> Open           $

In [35]:
aln.malign3d()

malign3_328_> Initial framework positions:      447
malign3_330_> Framework, Cycles, RMS_frw(i-1,i):        1       20        0.0000
malign3_332_> Framework:        1
# Sequence alignment of the structurally conserved regions
# [average distance and standard deviation are with respect
#  to the framework (i.e., average structure)]
#
#  N av ds st dv   5eqgA  4pypA  4zw9A  5c65A  4ybqA  4yb9D  4ldsA  4ja3A  4qiqA  4gbyA  6n3iA  6h7dA  
# =====================================================================================================
   1 1.537 0.532   A  19  A  19  A  17  A  17  A  23  F  26  G  12  A  17  A  17  A  17  A  17  A  32  
# =====================================================================================================
   2 0.938 0.364   S  23  S  23  S  21  S  21  S  28  S  29  G  16  G  21  G  21  G  21  G  21  G  36  
   3 0.665 0.303   L  24  L  24  F  22  F  22  F  29  F  30  L  17  L  22  L  22  L  22  L  22  L  37  
   4 0.768 0.297   Q  25  Q  25  Q  23  Q

In [32]:
aln.id_table(matrix_file='family.mat')
env.dendrogram(matrix_file='family.mat', cluster_cut=-1.0)


Sequence identity comparison (ID_TABLE):

   Diagonal       ... number of residues;
   Upper triangle ... number of identical residues;
   Lower triangle ... % sequence identity, id/min(length).

         5eqgA @24pypA @34zw9A @15c65A @24ybqA @34yb9D @34ldsA @34ja3A @34qiqA @34gbyA @26n3iA @36h7dA @2
5eqgA @2      447     444     188     170     106     167      94     107      88      72      71      64
4pypA @3       99     447     187     169     105     166      94     107      88      72      71      64
4zw9A @1       42      42     470     424     164     117      69      70      69     113     117     108
5c65A @2       38      38      93     457     168     108      63      67      65     111     113     110
4ybqA @3       24      23      36      37     452     176      64      54      56      94      98     100
4yb9D @3       38      38      27      25      40     437      85      90      74      61      60      67
4ldsA @3       22      22      16      15      15      20    

## Find Template Method 02

In [27]:
def findTemplate(spcode):
    name, outname, sequence = code2pirseq(spcode)
    # Descargar desde https://salilab.org/modeller/downloads/pdb_95.pir.gz
    pdb_95_pir = "pdb_95.pir.gz"
    seqQuery   = outname
    
    from modeller import environ, log, alignment, sequence_db
    
    log.verbose()
    env = environ()
    
    #-- Prepare the input files
    
    #-- Read in the sequence database
    sdb = sequence_db(env)
    sdb.read(seq_database_file=pdb_95_pir, seq_database_format='PIR',
             chains_list='ALL', minmax_db_seq_len=(30, 4000), clean_sequences=True)
    
    #-- Write the sequence database in binary form
    sdb.write(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
              chains_list='ALL')
    
    #-- Now, read in the binary database
    sdb.read(seq_database_file='pdb_95.bin', seq_database_format='BINARY',
             chains_list='ALL')
    
    #-- Read in the target sequence/alignment
    aln = alignment(env)
    aln.append(file=seqQuery, alignment_format='PIR', align_codes='ALL')
    
    #-- Convert the input sequence/alignment into
    #   profile format
    prf = aln.to_profile()
    
    #-- Scan sequence database to pick up homologous sequences
    prf.build(sdb, matrix_offset=-450, rr_file='${LIB}/blosum62.sim.mat',
              gap_penalties_1d=(-500, -50), n_prof_iterations=1,
              check_profile=False, max_aln_evalue=0.01)
    
    #-- Write out the profile in text format
    prf.write(file='%s_profile.prf' % name, profile_format='TEXT')
    
    #-- Convert the profile back to alignment format
    aln = prf.to_alignment()
    
    #-- Write out the alignment file
    aln.write(file='%s_profile.ali' % name, alignment_format='PIR')

In [29]:
spcode = "P11167"
findTemplate(spcode)

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL9v21}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v21}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4796159    4683.749     4.574

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4796687    4684.265     4.574
openf___224_> Open           ${MODINSTALL9v21}/modlib/resdih.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4845287    4731.726     4.621
rdrdih__263_> Number of dihedral angle types         :        9
              Maximal number of dihedral angle optima:        3
              Dihedral angle names                   :  Alph Phi Psi Omeg chi1 chi2 chi3 chi4 chi5
openf___224_> Open           ${MODINSTALL9v21}/modlib/radii.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:      4858587    4744.714     4.634
openf___224_> Open           $

In [30]:
%more GTR1_RAT_profile.ali


>P1;GTR1_RAT
sequence:GTR1_RAT:    0: :    0: :::-1.00:-1.00
MEPSSKKVTGRLMLAVGGAVLGSLQFGYNTGVINAPQKVIEEFYNQTWNHRYGESIPSTTLTTLWSLSVAIFSVG
GMIGSFSVGLFVNRFGRRNSMLMMNLLAFVSAVLMGFSKLGKSFEMLILGRFIIGVYCGLTTGFVPMYVGEVSPT
ALRGALGTLHQLGIVVGILIAQVFGLDSIMGNADLWPLLLSVIFIPALLQCILLPFCPESPRFLLINRNEENRAK
SVLKKLRGTADVTRDLQEMKEEGRQMMREKKVTILELFRSPAYRQPILIAVVLQLSQQLSGINAVFYYSTSIFEK
AGVQQPVYATIGSGIVNTAFTVVSLFVVERAGRRTLHLIGLAGMAGCAVLMTIALALLEQLPWMSYLSIVAIFGF
VAFFEVGPGPIPWFIVAELFSQGPRPAAVAVAGFSNWTSNFIVGMCFQYVEQLCGPYVFIIFTVLLVLFFIFTYF
KVPETKGRTFDEIASGFRQGGASQSDKTPEELFHPLGADSQV*

>P1;5c65A
structure:5c65A:    1: :  453: :::-1.00:-1.00
------KVTPALIFAITVATIGSFQFGYNTGVINAPEKIIKEFIQKTLTDK-GNAPPSVLLTSLWSLSVAIFSVG
GMIGSFSVGLFVNRFGRRNSMLIVNLLAVTGGCFMGLCKVAKSVEMLILGRLVIGLFCGLCTGFVPMYIGEISPT
ALRGAFGTLNQLGIVVGILVAQIFGLEFILGSEELWPLLLGFTILPAILQSAALPFCPESPRFLLINRKEEENAK
QILQRLWGTQDVSQDIQEMKDESARMSQEKQVTVLELFRVSSYRQPIIISIVLQLSQQLSGINAVFYYSTGIFKD
A----PIYATIGAGVVNTIFTVVSLFLVERAGRRTLHMIGLGGMAFCSTLMTVSLLLKD----MSFVCIGAILVF
V

## Cleaning the Folder

In [6]:
import os,datetime, shutil
archivos = [x for x in os.listdir(".") if (x.count(".ipynb")==0 and os.path.isdir(x)==False)]
dt = str(datetime.datetime.now())
folder_file = "01_Template_Search_%s" % dt
os.mkdir(folder_file)
for f in archivos:
    shutil.move(f, folder_file)